In [157]:
### IMPORTS ###
from SPARQLWrapper import SPARQLWrapper, JSON, TURTLE, POST, GET
from pprint import pprint
import csv

In [158]:
### QUERIES ###

def get_closure_count(endpoint):
    endpoint.query.resetQuery()
    endpoint.query.clearParameter('reasoning')
    endpoint.query.addParameter('reasoning', 'true')
    endpoint.query.setReturnFormat(JSON)
    endpoint.update.setMethod(GET)
    endpoint.query.setQuery("""
        SELECT (COUNT(*) as ?closure_count) WHERE {
            ?sub ?pred ?obj .
            FILTER(
                !REGEX(STR(?sub), 'http://www.w3.org/2002/07/owl') &&
                !REGEX(STR(?pred), 'http://www.w3.org/2002/07/owl') &&
                !REGEX(STR(?obj), 'http://www.w3.org/2002/07/owl')
            )
        }
    """)
    result = endpoint.query.query().convert()['results']['bindings'][0]
    return result['closure_count']['value']

def get_closure_turtle(endpoint):
    endpoint.query.resetQuery()
    endpoint.query.clearParameter('reasoning')
    endpoint.query.addParameter('reasoning', 'true')
    endpoint.update.setMethod(GET)
    endpoint.query.setQuery("""
        CONSTRUCT{
            ?sub ?pred ?obj
        } WHERE {
            ?sub ?pred ?obj .
            FILTER(
                !REGEX(STR(?sub), 'http://www.w3.org/2002/07/owl') &&
                !REGEX(STR(?pred), 'http://www.w3.org/2002/07/owl') &&
                !REGEX(STR(?obj), 'http://www.w3.org/2002/07/owl')
            )
        }
    """)
    result = endpoint.query.query().convert()
    return result.serialize(format='nt').decode('utf-8')

def remove_old_data(endpoint):
    endpoint.update.resetQuery()
    endpoint.update.setMethod(POST)
    delete_query = """
        DELETE WHERE {
            ?sub ?pred ?obj .
        }
    """
    endpoint.update.setQuery(delete_query)
    delete_result = endpoint.update.query()
    assert int(get_closure_count(endpoint)) == 0
    return delete_result

def add_new_data(endpoint, data):
    endpoint.update.resetQuery()
    endpoint.update.setReturnFormat(JSON)
    endpoint.update.setMethod(POST)
    update_query = """
        INSERT DATA {{
            {}
        }}
    """.format(data)
    endpoint.update.setQuery(update_query)
    update_result = endpoint.update.query()
    
def remove_axiom(endpoint, axiom):
    endpoint.update.resetQuery()
    endpoint.update.setMethod(POST)
    delete_query = """
        DELETE WHERE {{
            {sub} {pred} {obj} .
        }}
    """.format(
            sub=to_valid_iri(axiom['sub']['value']),
            pred=to_valid_iri(axiom['pred']['value']), 
            obj=to_valid_iri(axiom['obj']['value'])
    )
    endpoint.update.setQuery(delete_query)
    delete_result = endpoint.update.query()
    return delete_result

def get_data_count(endpoint):
    endpoint.query.resetQuery()
    endpoint.query.clearParameter('reasoning')
    endpoint.query.addParameter('reasoning', 'false')
    endpoint.query.setReturnFormat(JSON)
    endpoint.update.setMethod(GET)
    endpoint.query.setQuery("""
        SELECT (COUNT(*) AS ?data_count ) WHERE {
            ?sub ?pred ?obj .
            FILTER(
                !REGEX(STR(?sub), 'http://www.w3.org/2002/07/owl') &&
                !REGEX(STR(?pred), 'http://www.w3.org/2002/07/owl') &&
                !REGEX(STR(?obj), 'http://www.w3.org/2002/07/owl')
            )
        }
    """)
    result = endpoint.query.query().convert()['results']['bindings'][0]
    return result['data_count']['value']
    
def reset_data(endpoint, axioms):
    remove_old_data(endpoint)
    add_new_data(endpoint, axioms)

In [159]:
### HELPERS ###

def to_valid_iri(iri): 
    return '<' + iri.strip() + '>'

def get_recursive_closure_count(endpoint, old_count=0):
    count = get_closure_count(endpoint)
    if count == old_count:
        return count
    else:
        closure = get_closure_turtle(endpoint)
        remove_old_data(endpoint)
        add_new_data(endpoint, closure)
        return get_recursive_closure(endpoint, count)

In [160]:
### CONFIGURE CSV WRITER ###
file = open('rdfs_closure_counts.csv', mode='w', newline='')
fieldnames = ['excluded_axiom', 'closure_count', 'data_count', 'redundant']
writer = csv.DictWriter(file, fieldnames=fieldnames)
writer.writeheader()
file.flush()

In [161]:
### CONFIGURE SPARQL ENDPOINT REQUESTS ###
class Endpoint:
    query  = SPARQLWrapper('http://localhost:5820/RDFS/query')
    update = SPARQLWrapper('http://localhost:5820/RDFS/update')
    
    def __init__(self):
        pass
endpoint = Endpoint()

In [162]:
### LOAD ALL AXIOMS ###

### AS JSON ###
endpoint.query.resetQuery()
endpoint.query.clearParameter('reasoning')
endpoint.query.addParameter('reasoning', 'false')
endpoint.query.setReturnFormat(JSON)
endpoint.query.setMethod(GET)
endpoint.query.setQuery("""
    SELECT ?sub ?pred ?obj WHERE {
        ?sub ?pred ?obj .
        FILTER(
            !REGEX(STR(?sub), 'http://www.w3.org/2002/07/owl') &&
            !REGEX(STR(?pred), 'http://www.w3.org/2002/07/owl') &&
            !REGEX(STR(?obj), 'http://www.w3.org/2002/07/owl')
        )
    }
""")
axioms = endpoint.query.query().convert()['results']['bindings']
axioms_count = len(axioms)

### AS NTUPLES ###
endpoint.query.resetQuery()
endpoint.query.clearParameter('reasoning')
endpoint.query.addParameter('reasoning', 'false')
endpoint.query.setMethod(GET)
endpoint.query.setQuery("""
    CONSTRUCT {
        ?sub ?pred ?obj .
    } WHERE {
        ?sub ?pred ?obj .
        FILTER(
            !REGEX(STR(?sub), 'http://www.w3.org/2002/07/owl') &&
            !REGEX(STR(?pred), 'http://www.w3.org/2002/07/owl') &&
            !REGEX(STR(?obj), 'http://www.w3.org/2002/07/owl')
        )
    }
""")
axioms_data = endpoint.query.query().convert().serialize(format='nt').decode('utf-8')

In [163]:
### CALCULATE FULL RECURSIVE CLOSURE WITH ONE EXCLUDED AXIOM ###
for axiom in axioms:
    # RESET DATA WITH ONLY AXIOMS #
    reset_data(endpoint, axioms_data)
    
    # GET RECURSIVE FULL CLOSURE COUNT #
    full_recursive_closure_count = get_recursive_closure_count(endpoint)
    
    # RESET DATA WITH ONLY AXIOMS #
    reset_data(endpoint, axioms_data)
    
    # REMOVE ONE AXIOM #
    remove_axiom(endpoint, axiom)
    
    # GET DATA COUNT #
    new_data_count = get_data_count(endpoint)
    
    # GET RECURSIVE NEW CLOSURE COUNT #
    new_recursive_closure_count = get_recursive_closure_count(endpoint)
    
    # FLAG REDUNDANT AXIOMS #
    redudant = new_recursive_closure_count == full_recursive_closure_count

    # RECORD TRANSACTION #
    transaction = {
        'excluded_axiom': '{sub} {pred} {obj}'.format(
            sub=to_valid_iri(axiom['sub']['value']),
            pred=to_valid_iri(axiom['pred']['value']), 
            obj=to_valid_iri(axiom['obj']['value'])
        ),
        'closure_count': new_recursive_closure_count,
        'data_count': new_data_count,
        'redundant': redudant
    }
    pprint(transaction)
    writer.writerow(transaction)
file.close()

{'closure_count': '117',
 'data_count': '49',
 'excluded_axiom': '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> '
                   '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> '
                   '<http://www.w3.org/1999/02/22-rdf-syntax-ns#Property>',
 'redundant': True}
{'closure_count': '86',
 'data_count': '49',
 'excluded_axiom': '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> '
                   '<http://www.w3.org/2000/01/rdf-schema#domain> '
                   '<http://www.w3.org/2000/01/rdf-schema#Resource>',
 'redundant': False}
{'closure_count': '116',
 'data_count': '49',
 'excluded_axiom': '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> '
                   '<http://www.w3.org/2000/01/rdf-schema#range> '
                   '<http://www.w3.org/2000/01/rdf-schema#Class>',
 'redundant': False}
{'closure_count': '117',
 'data_count': '49',
 'excluded_axiom': '<http://www.w3.org/1999/02/22-rdf-syntax-ns#subject> '
                   '<http://www.w3.org/1999/

{'closure_count': '116',
 'data_count': '49',
 'excluded_axiom': '<http://www.w3.org/2000/01/rdf-schema#subPropertyOf> '
                   '<http://www.w3.org/2000/01/rdf-schema#domain> '
                   '<http://www.w3.org/1999/02/22-rdf-syntax-ns#Property>',
 'redundant': False}
{'closure_count': '116',
 'data_count': '49',
 'excluded_axiom': '<http://www.w3.org/2000/01/rdf-schema#subPropertyOf> '
                   '<http://www.w3.org/2000/01/rdf-schema#range> '
                   '<http://www.w3.org/1999/02/22-rdf-syntax-ns#Property>',
 'redundant': False}
{'closure_count': '116',
 'data_count': '49',
 'excluded_axiom': '<http://www.w3.org/2000/01/rdf-schema#subClassOf> '
                   '<http://www.w3.org/2000/01/rdf-schema#domain> '
                   '<http://www.w3.org/2000/01/rdf-schema#Class>',
 'redundant': False}
{'closure_count': '116',
 'data_count': '49',
 'excluded_axiom': '<http://www.w3.org/2000/01/rdf-schema#subClassOf> '
                   '<http://www.w3.or

limit_output extension: Maximum message size of 10000 exceeded with 10102 characters